## HYPEROPT / optional task for the student
 extend WandB.ai integration in the code with sweeps
 (e.g. add variables like learning rate, optimizer, neurons_FC1, neurons_FC2)
 help: https://docs.wandb.ai/guides/sweeps and
       https://github.com/wandb/wandb/issues/5003
 store the hyperparameters and val_acc to wandb
 run at least 20 trainings
 in wandb.ai, export the result of the runs as a .csv file,
 in wandb.ai, create a report from the sweep results, share by public link,

In [1]:
!pip install pytorch-lightning --quiet

In [2]:
!pip install wandb --quiet

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import CIFAR10

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

import wandb

In [4]:
class CIFAR10DataModule(pl.LightningDataModule):
    '''organize the data pipeline from accessing the data to loading it using PyTorch dataloaders'''


    def __init__(self, batch_size, data_dir: str = './'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        self.dims = (3, 32, 32)
        self.num_classes = 10

    def prepare_data(self):
        # download the CIFAR-10 dataset
        CIFAR10(self.data_dir, train=True, download=True)
        CIFAR10(self.data_dir, train=False, download=True)

 # PyTorch dataset instances
    def setup(self, stage=None):

        if stage == 'fit' or stage is None:
            cifar_full = CIFAR10(self.data_dir, train=True, transform=self.transform)
            self.cifar_train, self.cifar_val = random_split(cifar_full, [45000, 5000])

        if stage == 'test' or stage is None:
            self.cifar_test = CIFAR10(self.data_dir, train=False, transform=self.transform)

    # dataloaders
    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size)

In [5]:
class CIFAR10LitModel(pl.LightningModule):
    '''model architecture, training, testing and validation loops'''
    def __init__(self, input_shape, num_classes, learning_rate=3e-4):
        super().__init__()

        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate

        # model architecture
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1)
        self.conv4 = nn.Conv2d(64, 64, 3, 1)

        self.pool1 = torch.nn.MaxPool2d(2)
        self.pool2 = torch.nn.MaxPool2d(2)

        n_sizes = self._get_output_shape(input_shape)

        # linear layers for classifier head
        ## Modify the architecture for adding the layer with 256 neurons
        self.fc1 = nn.Linear(n_sizes, 512)
        self.fc2 = nn.Linear(512, 256)  # <-- Existing layer, but now connecting to the new layer
        self.fc3 = nn.Linear(256, 128)  # <-- New layer with 256 neurons
        self.fc4 = nn.Linear(128, num_classes)  # <-- Adjusted this layer

        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.train_acc=Accuracy(task='multiclass',num_classes=10)
        self.val_acc=Accuracy(task='multiclass',num_classes=10)
        self.test_acc=Accuracy(task='multiclass',num_classes=10)

    def _get_output_shape(self, shape):
        '''returns the size of the output tensor from the conv layers'''

        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._feature_extractor(input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    # computations
    # conv1,relu, conv2,relu, maxpool,conv3,relu,conv4,relu,maxpool
    def _feature_extractor(self, x):
        '''extract features from the conv blocks'''
        x = F.relu(self.conv1(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
        return x

    def forward(self, x):
       '''produce final model output'''
       x = self._feature_extractor(x)
       x = x.view(x.size(0), -1)
       x = F.relu(self.fc1(x))
       x = F.relu(self.fc2(x))
       x = F.relu(self.fc3(x)) # we add this line cause the new added layer
       x = F.log_softmax(self.fc4(x), dim=1) # We adjust this line for the new layer
       return x

    # train loop
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)

        # metric
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        return loss

    # validation loop
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    # test loop
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('test_loss', loss, on_epoch=True) #
        self.log('test_acc', acc, on_epoch=True) #
        return loss

    # optimizers
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), self.learning_rate) #
        return optimizer

In [6]:
# class for visualizing one batch of validation images along with predicted and class label
class ImagePredictionLogger(pl.Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = self.val_labels[:num_samples]

    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, 1)

        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                            for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
            })

In [7]:
cifar = CIFAR10DataModule(batch_size=32)
cifar.prepare_data()
cifar.setup()
# grab samples to log predictions on
samples = next(iter(cifar.val_dataloader()))


Files already downloaded and verified
Files already downloaded and verified


In [8]:
### WandB
def train_model(learning_rate=1e-3):
    wandb.login(key='78c0a600cbd2a43b80396717ef10ee93a8129894')
    config=wandb.config
    wandb_logger = WandbLogger(project='2.3 practice (hyperparameter optimization)', job_type='train', log_model="all")
    # instantiate classes
    dm = CIFAR10DataModule(32)
    dm.prepare_data()
    dm.setup()
    model = CIFAR10LitModel((3, 32, 32), dm.num_classes, learning_rate)
    wandb_logger.watch(model)
    # Initialize Callbacks
    checkpoint_callback = pl.callbacks.ModelCheckpoint()
    early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_acc", patience=3, verbose=False, mode="max")
    ### WandB
    trainer = pl.Trainer(max_epochs=5,
                     logger=wandb_logger,
                     callbacks=[checkpoint_callback, early_stop_callback,ImagePredictionLogger(samples)]
                    )
    # Train the model
    trainer.fit(model, dm)

    # Evaluate the model
    trainer.test(dataloaders=cifar.test_dataloader())
    # tell the WandB you have finished
    wandb.finish()

In [9]:
train_model()

wandb: Currently logged in as: acuadrosrivas (alecuadrosrivas). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Files already downloaded and verified
Files already downloaded and verified


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name      | Type               | Params
--------------------------------------------------
0  | conv1     | Conv2d             | 896   
1  | conv2     | Conv2d             | 9.2 K 
2  | conv3     | Conv2d             | 18.5 K
3  | conv4     | Conv2d             | 36.9 K
4  | pool1     | MaxPool2d          | 0     
5  | pool2     | MaxPool2d          | 0     
6  | fc1       | Linear             | 819 K 
7  | fc2       | Linear             | 131 K 
8  | fc3       | Linear             | 32.9 K
9  | fc4       | Linear             | 1.3 K 
10 | accuracy  | MulticlassAccuracy | 0     
11 | train_acc | MulticlassAccuracy | 0     
12 | val_acc   | MulticlassAccuracy | 0     
13 | test_acc  | MulticlassAccuracy | 0     
--------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.203     T

...

Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at ./2.3 practice (hyperparameter optimization)/nsbwmmbx/checkpoints/epoch=4-step=7035.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at ./2.3 practice (hyperparameter optimization)/nsbwmmbx/checkpoints/epoch=4-step=7035.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7305999994277954     │
│         test_loss         │    0.8187376260757446     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
global_step,▁▂▄▅▇█
test_acc,▁
test_loss,▁
train_acc_epoch,▁▅▆▇█
train_acc_step,▁▂▄▄▄▄▅▄▅▅▅▅▅▅▅▆▅▇▆▇▆▆▅▇▇▅▅▆█▇▆▇▇▇▇▇▇█▇▇
train_loss_epoch,█▄▃▂▁
train_loss_step,██▆▆▅▅▅▅▄▅▄▅▄▅▅▃▄▂▃▃▃▂▃▂▂▃▃▄▂▃▃▁▂▂▂▂▂▁▁▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
val_acc,▁▅▇▇█
val_loss,█▃▂▂▁


In [10]:
### HYPEROPT / optional task for the student
# extend WandB.ai integration in the code with sweeps
# (e.g. add variables like learning rate, optimizer, neurons_FC1, neurons_FC2)
# help: https://docs.wandb.ai/guides/sweeps and
#       https://github.com/wandb/wandb/issues/5003
# store the hyperparameters and val_acc to wandb
# run at least 20 trainings
# in wandb.ai, export the result of the runs as a .csv file,
# in wandb.ai, create a report from the sweep results, share by public link,

In [11]:
sweep_config = {
  'name': 'my-sweep',
  'method': 'random',
  'metric': {
  'goal': 'maximize',
  'name': 'val_acc'
  },
    'parameters': {
      'learning_rate': {
        'min': 1e-4,
        'max': 1e-2
      },
      'optimizer': {
        'values': ['adam', 'sgd']
      },
      'neurons_FC1': {
        'min': 100,
        'max': 1000
      },
      'neurons_FC2': {
        'min': 50,
        'max': 500
      }
  }
}

In [12]:
sweep_id = wandb.sweep(sweep_config, project="2.3 practice (hyperparameter optimization)")

Create sweep with ID: 8p1anj8o
Sweep URL: https://wandb.ai/alecuadrosrivas/2.3%20practice%20%28hyperparameter%20optimization%29/sweeps/8p1anj8o


In [13]:
class CIFAR10LitModel(pl.LightningModule):
    '''model architecture, training, testing and validation loops'''
    def __init__(self, input_shape, num_classes, learning_rate=3e-4, optimizer_type='adam', neurons_FC1=512, neurons_FC2=256):
        super(CIFAR10LitModel, self).__init__()

        # log hyperparameters
        self.save_hyperparameters()
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.optimizer_type = optimizer_type

        # model architecture
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1)
        self.conv4 = nn.Conv2d(64, 64, 3, 1)

        self.pool1 = torch.nn.MaxPool2d(2)
        self.pool2 = torch.nn.MaxPool2d(2)

        n_sizes = self._get_output_shape(input_shape)

        # linear layers for classifier head
        ## Modify the architecture for adding the layer with 256 neurons
        self.fc1 = nn.Linear(n_sizes, neurons_FC1)
        self.fc2 = nn.Linear(neurons_FC1, neurons_FC2)  # <-- Existing layer, but now connecting to the new layer
        self.fc3 = nn.Linear(neurons_FC2, 128)  # <-- New layer with 256 neurons
        self.fc4 = nn.Linear(128, num_classes)  # <-- Adjusted this layer

        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.train_acc=Accuracy(task='multiclass',num_classes=10)
        self.val_acc=Accuracy(task='multiclass',num_classes=10)
        self.test_acc=Accuracy(task='multiclass',num_classes=10)

    def _get_output_shape(self, shape):
        '''returns the size of the output tensor from the conv layers'''

        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._feature_extractor(input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    # computations
    # conv1,relu, conv2,relu, maxpool,conv3,relu,conv4,relu,maxpool
    def _feature_extractor(self, x):
        '''extract features from the conv blocks'''
        x = F.relu(self.conv1(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
        return x

    def forward(self, x):
       '''produce final model output'''
       x = self._feature_extractor(x)
       x = x.view(x.size(0), -1)
       x = F.relu(self.fc1(x))
       x = F.relu(self.fc2(x))
       x = F.relu(self.fc3(x)) # we add this line cause the new added layer
       x = F.log_softmax(self.fc4(x), dim=1) # We adjust this line for the new layer
       return x

    # train loop
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)

        # metric
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        return loss

    # validation loop
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    # test loop
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('test_loss', loss, on_epoch=True) #
        self.log('test_acc', acc, on_epoch=True) #
        return loss

    # optimizers
    def configure_optimizers(self):
        if self.optimizer_type == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), self.learning_rate)
        elif self.optimizer_type == 'sgd':
            optimizer = torch.optim.SGD(self.parameters(), self.learning_rate)
        return optimizer

In [14]:
### WandB
def train_model(learning_rate=1e-3, optimizer_type='adam', neurons_FC1=512, neurons_FC2=256):

    wandb.login(key='78c0a600cbd2a43b80396717ef10ee93a8129894')

    with wandb.init(project='2.3 practice (hyperparameter optimization)',job_type='train',config={
      'learning_rate': learning_rate,
      'optimizer': optimizer_type,
      'neurons_FC1': neurons_FC1,
      'neurons_FC2': neurons_FC2
    }) as run:
      config = wandb.config

    # Configurations
      learning_rate = config.learning_rate
      optimizer_type = config.optimizer
      neurons_FC1 = config.neurons_FC1
      neurons_FC2 = config.neurons_FC2
      wandb_logger = WandbLogger(project='2.3 practice (hyperparameter optimization)', job_type='train', log_model="all")

    # instantiate classes
      dm = CIFAR10DataModule(32)
      dm.prepare_data()
      dm.setup()
      model = CIFAR10LitModel((3, 32, 32), dm.num_classes, learning_rate, optimizer_type, neurons_FC1, neurons_FC2)
      wandb_logger.watch(model)

    # Initialize Callbacks
      checkpoint_callback = pl.callbacks.ModelCheckpoint()
      early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_acc", patience=3, verbose=False, mode="max")

    ### WandB
      trainer = pl.Trainer(max_epochs=5,
                      logger=wandb_logger,
                      callbacks=[checkpoint_callback, early_stop_callback,ImagePredictionLogger(samples)]
                      )
    # Train the model
      trainer.fit(model, dm)

    # Evaluate the model
      trainer.test(dataloaders=cifar.test_dataloader())

    # tell the WandB you have finished
      wandb.finish()

train_model()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name      | Type               | Params
--------------------------------------------------
0  | conv1     | Conv2d             | 896   
1  | conv2     | Conv2d             | 9.2 K 
2  | conv3     | Conv2d             | 18.5 K
3  | conv4     | Conv2d             | 36.9 K
4  | pool1     | MaxPool2d          | 0     
5  | pool2     | MaxPool2d          | 0     
6  | fc1       | Linear             | 819 K 
7  | fc2       | Linear             | 131 K 
8  | fc3       | Linear             | 32.9 K
9  | fc4       | Linear             | 1.3 K 
10 | accuracy  | MulticlassAccuracy | 0     
11 | train_acc | MulticlassAccuracy | 0     
12 | val_acc   | MulticlassAccuracy | 0     
13 | test_acc  | MulticlassAccuracy | 0     
--------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.203     T

...

Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at ./2.3 practice (hyperparameter optimization)/efavp3wl/checkpoints/epoch=4-step=7035.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at ./2.3 practice (hyperparameter optimization)/efavp3wl/checkpoints/epoch=4-step=7035.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7019000053405762     │
│         test_loss         │    0.8739820122718811     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
global_step,▁▂▄▅▇█
test_acc,▁
test_loss,▁
train_acc_epoch,▁▄▆▇█
train_acc_step,▁▂▂▂▃▃▄▅▅▄▄▃▅▅▅▅▇▆▅▅▆▅▄▇▆▆▇▆█▆▆▅▆█▆▇▆█▇█
train_loss_epoch,█▅▃▂▁
train_loss_step,█▇▇▇▇█▆▅▄▆▅█▅▅▄▅▄▃▄▄▃▄▅▃▃▃▃▃▁▃▃▃▃▁▃▂▃▂▃▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▅▇██
val_loss,█▄▂▁▁


In [15]:
wandb.agent(sweep_id, train_model, count=20)


wandb: Agent Starting Run: 5jpni55g with config:
wandb: 	learning_rate: 0.006408704427911986
wandb: 	neurons_FC1: 340
wandb: 	neurons_FC2: 297
wandb: 	optimizer: adam
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Files already downloaded and verified
Files already downloaded and verified


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name      | Type               | Params
--------------------------------------------------
0  | conv1     | Conv2d             | 896   
1  | conv2     | Conv2d             | 9.2 K 
2  | conv3     | Conv2d             | 18.5 K
3  | conv4     | Conv2d             | 36.9 K
4  | pool1     | MaxPool2d          | 0     
5  | pool2     | MaxPool2d          | 0     
6  | fc1       | Linear             | 544 K 
7  | fc2       | Linear             | 101 K 
8  | fc3       | Linear             | 38.1 K
9  | fc4       | Linear             | 1.3 K 
10 | accuracy  | MulticlassAccuracy | 0     
11 | train_acc | MulticlassAccuracy | 0     
12 | val_acc   | MulticlassAccuracy | 0     
13 | test_acc  | MulticlassAccuracy | 0     
--------------------------------------------------
750 K     Trainable params
0         Non-trainable params
750 K     Total params
3.002     T

...

Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at ./2.3 practice (hyperparameter optimization)/3973hz5o/checkpoints/epoch=4-step=7035.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'neurons_FC1' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'neurons_FC2' was locked by 'sweep' (ignored update).
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at ./2.3 practice (hyperparameter optimization)/3973hz5o/checkpoints/epoch=4-step=7035.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.16779999434947968    │
│         test_loss         │     2.299128293991089     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
global_step,▁▂▄▅▇█
test_acc,▁
test_loss,▁
train_acc_epoch,▁▂▂▄█
train_acc_step,▅▄▄▂▂▂▂▅▂▄▂▄▂▄▃▄▅▅▃▂▄▁▃▃▅▂▅▃█▃▅▆▅▅▂▄▃▄▃▅
train_loss_epoch,█▆▆▅▁
train_loss_step,▁▄▄█▄▇▄▆▅▃▅▄▅▄▄▄▄▄▃▄▄▅▄▄▄▄▄▄▃▅▃▄▄▄▄▄▄▃▃▃
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▁▁▁█
val_loss,██▇▆▁
